# Code Translator that compares different frontier and open source models to determine which are most efficient to compare busineess cost allowing users to run the converted code on their own machine to compare run times

Utilizes:
    - Python code execution
    - System spec reading
    - Subprocesses

In [2]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display
from system_info import retrieve_system_info, rust_toolchain_info, go_toolchain_info

In [ ]:
# Initialize env variables

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set")

In [4]:
# Initialize clients

openai = OpenAI()
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)


In [5]:
# Define your models

models = ["gpt-5", "claude-sonnet-4-5-20250929", "grok-4", "gemini-2.5-pro", "qwen2.5-coder", "deepseek-coder-v2", "gpt-oss:20b", "qwen/qwen3-coder-30b-a3b-instruct", "openai/gpt-oss-120b", "llama3.2" ]

clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic, "grok-4": grok, "gemini-2.5-pro": gemini, "openai/gpt-oss-120b": groq, "qwen2.5-coder": ollama, "deepseek-coder-v2": ollama, "gpt-oss:20b": ollama, "qwen/qwen3-coder-30b-a3b-instruct": openrouter, "llama3.2" : ollama}


In [26]:
# Coding languages

system_info = retrieve_system_info()
rust_info = rust_toolchain_info()
go_info = go_toolchain_info()

languageExtension = {"cpp": "cpp", "rust": "rs", "go": "go" }
languages = ["cpp", "rust", "go" ]
toolchain = {"cpp": 'N/A' , "rust": rust_info, "go": go_info }


In [7]:
compile_command = [
    "/Users/gidd/.cargo/bin/rustc",
    "main.rs",
    "-C", "opt-level=3",
    "-C", "target-cpu=native",
    "-C", "codegen-units=1",
    "-C", "lto=fat",
    "-C", "panic=abort",
    "-C", "strip=symbols",
    "-o", "main",
]

run_command = ["./main"]

In [ ]:
# Check System config

def checkSystem(language):
    info = toolchain[language]
    extension = languageExtension[language]

    message = f"""
    Here is a report of the system information for my computer.
    I want to run a {language} compiler to compile a single {language} file called main.{extension} and then execute it in the simplest way possible.
    Please reply with whether I need to install a {language} toolchain to do this. If so, please provide the simplest step by step instructions to do so.

    If I'm already set up to compile {language} code, then I'd like to run something like this in Python to compile and execute the code:
    ```python
    compile_command = # something here - to achieve the fastest possible runtime performance
    compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
    run_command = # something here
    run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
    return run_result.stdout
    ```
    Please tell me exactly what I should use for the compile_command and run_command.
    Have the maximum possible runtime performance in mind; compile time can be slow. Fastest possible runtime performance for this platform is key.
    Reply with the commands in array format like:

    Compile Command
    {compile_command}

    Run Command
    {run_command}
    
    System information:
    {system_info}


    {f'{language} toolchain information:\n{info}' if language != 'cpp' else ''}
    """

    response = openai.chat.completions.create(model=models[0], messages=[{"role": "user", "content": message}])
    return response.choices[0].message.content
    #response = ollama.chat.completions.create(model=models[9], messages=[{"role": "user", "content": message}])
    #display(Markdown(response.choices[0].message.content))


# Edit compile command for your system and chosen language

In [13]:
# compile command example

compile_command = [
    "/Users/gidd/.cargo/bin/rustc",
    "main.rs",
    "-C", "opt-level=3",
    "-C", "target-cpu=native",
    "-C", "codegen-units=1",
    "-C", "lto=fat",
    "-C", "panic=abort",
    "-C", "strip=symbols",
    "-o", "main",
]

run_command = ["./main"]


In [14]:
def get_messages(python, language):
    system_prompt = f"""
    Your task is to convert Python code into high performance {language} code.
    Respond only with {language} code. Do not provide any explanation other than occasional comments.
    The {language} response needs to produce an identical output in the fastest possible time.
    """

    user_prompt = f"""
    Port this Python code to {language} with the fastest possible implementation that produces identical output in the least time.
    The system information is:
    {system_info}
    Your response will be written to a file called main.{language} and then compiled and executed; the compilation command is:
    {compile_command}
    Respond only with {language} code.
    Python code to port:

    ```python
    {python}
    """

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [15]:
def write_output(code, language):
    extension = languageExtension[language]
    with open(f"main.{extension}", "w") as f:
        f.write(code)

In [16]:
def portCode(model, python, language):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=get_messages(python, language), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```rust','').replace('```go','').replace('```','')
    return reply

In [17]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [18]:
def compile_and_run(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [19]:
# example python code

python_hard = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
# gradio

def update_get_compile(language):
    return gr.Button(value=f"Generate compile command for {language} to replace command_compile")

def update_code_output(language):
    return gr.Code(label=f"{language} (generated)")

def update_convert_output(language):
    return gr.Button(value=f"Port to {language}")

def update_code_run_output(language):
    return gr.Button(value=f"Run {language}")

def update_code_out_output(language):
    return gr.TextArea(label=f"{language} result")

with gr.Blocks(title=f"Port code from Python to selected language") as interface:
    with gr.Row(equal_height=True):
        with gr.Column(scale=2):
            language = gr.Dropdown(languages, value=languages[0], show_label=False)
            compile_code = gr.TextArea(label="Compile Command", lines=6)
            get_compile = gr.Button(f"Generate compile command for {language.value} to replace command_compile")
            language.change(fn=update_get_compile, inputs=language, outputs=get_compile)

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=python_hard,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            code_output= gr.Code(
                label=f"{languages[0]} (generated)",
                value="",
                language="cpp", #rust and go not supported
                lines=26
            )
            language.change(fn=update_code_output, inputs=language, outputs=code_output)
    with gr.Row(equal_height=True):
        python_run = gr.Button("Run Python")
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"Port to {languages[0]}")
        code_run = gr.Button(f"Run {languages[0]}")
        language.change(fn=update_convert_output, inputs=language, outputs=convert)
        language.change(fn=update_code_run_output, inputs=language, outputs=code_run)

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8)
        with gr.Column(scale=6):
            code_out = gr.TextArea(label=f"{languages[0]} result", lines=8)
            language.change(fn=update_code_out_output, inputs=language, outputs=code_out)
       
    get_compile.click(fn=checkSystem, inputs=[language], outputs=[compile_code])
    convert.click(fn=portCode, inputs=[model, python, language], outputs=[code_output])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    code_run.click(fn=compile_and_run, inputs=[code_output], outputs=[code_out])
    


interface.launch(inbrowser=True)